In [1]:
#!pip3 install --user unidecode

# On-Machine Text Preprocessing

 1. Read in a document
 2. Tokenize it
 3. Bind the results together into one gigantic data frame

Do this with gensim: first, use their utils for stopwords, etc. next, build into a corpus type

In [2]:
%matplotlib inline

import os
import gensim as gs
import gzip
import itertools as it
import pandas as pd
import random
import re

from concurrent import futures

Yield all documents:

In [3]:
def collect_documents(root):
    for pwd, dirs, files in os.walk(root):
        documents = [doc for doc in files if doc.endswith('.txt.gz')]
        if not documents:
            continue
        for doc in documents:
            yield os.path.join(os.path.abspath(pwd), doc)

In [4]:
%%time
len(list(collect_documents('../../data/arxiv/text/')))

CPU times: user 8.13 s, sys: 716 ms, total: 8.84 s
Wall time: 8.76 s


1062145

Ingest and tokenize each document in parallel.

In [5]:
def ingest(path):
    identifier = os.path.basename(path).replace('.txt.gz', '')
    with gzip.GzipFile(path) as infil:
        return {'id': identifier, 'text': infil.read().decode()}

In [6]:
# collapse commonly-hyphenated words
HYPHEN_COLLAPSE = {
    r'x-ray': 'xray',
    r'e-mail': 'email',
    r'non-([a-zA-Z])': 'non\\1',
    r'space-time': 'spacetime'
}

RE_HYPHENS = {
    HYPHEN_COLLAPSE[entry]: re.compile(entry)
    for entry in HYPHEN_COLLAPSE
}

def tokenize(doc, disable_hyphen_split=False):
    # split on whitespace and remove empty tokens
    stream = filter(None, re.split(r'[\s]+', doc['text']))
    
    # normalize to ascii and convert to lowercase
    stream = [gs.corpora.textcorpus.deaccent(s).lower() for s in stream]
    
    for k in RE_HYPHENS:
        stream = [RE_HYPHENS[k].sub(k, s) for s in stream]
    
    # Remove bracket detritus like blah[blah]
    stream = [s for s in stream if re.search(r'[^\[\]]+[[^\[\]]+', s) is None]
    
    # Deeper split: hyphens, slashes, parens, braces, etc.
    # This throws away some technical information [e.g.: the "so(n)" lie algebra] but helps w/words
    if not disable_hyphen_split:
        # https://stackoverflow.com/a/952952/656740
        stream = filter(None, it.chain.from_iterable(re.split(r'[-/\\(){}–—,]+', s) for s in stream))
    
    # remove common LaTeX labels like fig:blah or eq:blah1
    stream = [s for s in stream if re.search(r'(fig|eqn?)[:a-zA-Z0-9]', s) is None]
    
    # strip edge punctuation
    stream = [re.sub(r'(?<=^)[^a-zA-Z0-9]+|[^a-zA-Z0-9]+(?=$)', '', s) for s in stream]
    
    # do not include nonword tokens
    stream = filter(lambda s: re.search(r'[a-zA-Z]', s), stream)
    
    # remove email addresses
    stream = [s for s in stream if s.find('@') == -1]
    
    # strip possession
    stream = [re.sub(r"'s$", '', s) for s in stream]
    
    # remove leftover colon-y things like 'sec:my_great_section'
    stream = [s for s in stream if s.find(':') == -1]
    
    # Add to dictionary
    tokens = {}
    for tkn in stream:
        if tkn not in tokens:
            tokens[tkn] = 0
        tokens[tkn] += 1
    
    return pd.DataFrame([
        {'doc': doc['id'], 'token': token, 'count': tokens[token]}
        for token in tokens
    ])

In [7]:
def ingest_and_tokenize(path, *args, **kwargs):
    return tokenize(ingest(path), *args, **kwargs)

In [ ]:
%%time
random.seed(2**10)

jobs = []
with futures.ProcessPoolExecutor() as fexec:
    for doc in random.sample(list(collect_documents('../../data/arxiv/text')), k=500000):
        jobs += [fexec.submit(ingest_and_tokenize, doc, disable_hyphen_split=False)]
    futures.wait(jobs)

tokens = pd.concat(job.result() for job in jobs)

/home/ec2-user/.local/lib/python3.7/site-packages/ipykernel_launcher.py:25: FutureWarning: Possible nested set at position 9
/home/ec2-user/.local/lib/python3.7/site-packages/ipykernel_launcher.py:25: FutureWarning: Possible nested set at position 9
/home/ec2-user/.local/lib/python3.7/site-packages/ipykernel_launcher.py:25: FutureWarning: Possible nested set at position 9
/home/ec2-user/.local/lib/python3.7/site-packages/ipykernel_launcher.py:25: FutureWarning: Possible nested set at position 9
/home/ec2-user/.local/lib/python3.7/site-packages/ipykernel_launcher.py:25: FutureWarning: Possible nested set at position 9
/home/ec2-user/.local/lib/python3.7/site-packages/ipykernel_launcher.py:25: FutureWarning: Possible nested set at position 9
/home/ec2-user/.local/lib/python3.7/site-packages/ipykernel_launcher.py:25: FutureWarning: Possible nested set at position 9
/home/ec2-user/.local/lib/python3.7/site-packages/ipykernel_launcher.py:25: FutureWarning: Possible nested set at position 9


In [10]:
tokens

count          doc           token
0      322  math0108135             the
1        9  math0108135          bigger
2       16  math0108135          brauer
3       19  math0108135           group
4      174  math0108135              is
5        1  math0108135          really
6        1  math0108135             big
7        1  math0108135          stefan
8        3  math0108135         schroer
9        1  math0108135    introduction
10      10  math0108135    grothendieck
11       1  math0108135           asked
12       2  math0108135         whether
13      39  math0108135            each
14       3  math0108135         torsion
15      30  math0108135           class
16      88  math0108135              in
17      41  math0108135              on
18     139  math0108135               a
19      36  math0108135          scheme
20      10  math0108135           comes
21      20  math0108135            from
22      78  math0108135              an
23       8  math0108135         azumaya
24      19  math0108135         algebra
25      45  math0108135            this
26       1  math0108135           major
27      36  math0108135            open
28       2  math0108135         problem
29       3  math0108135          theory
..     ...          ...             ...
289      1    1301.6127     computation
290      1    1301.6127             one
291      1    1301.6127           hence
292      2    1301.6127           total
293      4    1301.6127          arrays
294      1    1301.6127  asymptotically
295      3    1301.6127             sum
296      1    1301.6127           sizes
297      1    1301.6127           taken
298      1    1301.6127           argue
299      2    1301.6127       induction
300      1    1301.6127            thus
301      1    1301.6127       necessary
302      1    1301.6127          future
303      1    1301.6127    computations
304      2    1301.6127           space
305      1    1301.6127            used
306      1    1301.6127         storing
307      1    1301.6127        succinct
308      1    1301.6127           prove
309      1    1301.6127          useful
310      1    1301.6127           later
311      1    1301.6127       improving
312      1    1301.6127         running
313      1    1301.6127          turned
314      1    1301.6127           least
315      1    1301.6127       represent
316      1    1301.6127       retrieved
317      1    1301.6127            rank
318      1    1301.6127         queries

[425236712 rows x 3 columns]

In [ ]:
# Test a few other things: anything with numbers that we want? anything one character wide? punctuation?

In [11]:
token_counts = (
    tokens.groupby('token')['count']
            .agg('sum')
            .to_frame()
            .reset_index()
            .sort_values('count', ascending=False)
)

In [13]:
token_counts.head()

token      count
3774895   the  198966394
2669285    of   94487350
444879    and   65151888
1804200    in   58621376
317768      a   53350124

In [22]:
# None of these look terribly important
token_counts[token_counts['token'].apply(lambda t: re.search(r'[0-9]', t) is not None)].head(n=50)

token   count
193095      2d  220582
227470      3d  186952
128651      1d  107104
4106168     x1   56574
250030      4d   49898
201140     2nd   46451
199420   2mass   36446
1019448     d3   34271
2004698     k1   33916
335192      a1   31790
226453      3c   28926
266908      5d   28918
2115087     l1   27595
2266065    m31   26950
2006038     k3   26599
1636360     h1   26446
338543      a2   24280
3233459     s1   24073
1381121  fig.1   23790
2264957     m2   23054
1016761     d0   22961
3236922     s2   22284
231783     3rd   22229
751282      c1   19986
561212      b2   19534
752788      c2   18390
1018681     d2   17669
135756     1st   17515
127066      1a   17436
1017697     d1   17423
559011      b1   17248
2266929     m5   16713
1381485  fig.2   16649
1020935     d5   16509
191559      2a   16327
2742206     p1   15700
341237      a3   15464
127524      1b   15376
252711     4th   14794
3227989     s0   14726
1487406     g1   14614
1020544     d4   14588
1333032     f1   14491
191999      2b   14473
1021397     d7   14451
2263610     m1   14019
3238694     s3   13659
1021198     d6   12948
2518491    n=2   12922
2117016     l2   12465

In [20]:
# Might be able to drop these, but maybe not
token_counts[token_counts['token'].apply(len) < 3].head(n=20)

token     count
2669285    of  94487350
1804200    in  58621376
317768      a  53350124
1862358    is  51423252
3814778    to  46163410
4034503    we  30947861
740649     by  20841315
512003     as  17448658
606676     be  16891216
2685068    on  14765126
526854     at  10434671
439599     an  10059032
1872956    it   9831283
1783914    if   7299130
2701592    or   4801434
1881067     j   4529052
2250965     m   4267861
3213754     s   3772875
3527130    so   3307002
1005905     d   3263854

In [18]:
# Yep, we could collapse punctuation if it doesn't wreck the stemming process
token_counts[token_counts['token'].apply(lambda t: gs.parsing.preprocessing.RE_PUNCT.search(t) is not None)]

token    count
1762570                                           i.e  1832843
1193823                                           e.g   978287
510280                                      arxiv.org    58866
328595                                            a.s    57309
322115                                            a.e    57277
4013693                                         w.r.t    56115
1187117                                    dx.doi.org    48816
1763589                                         i.i.d    45675
1893179                                           j.m    42207
1882161                                           j.a    36078
319277                                            a.a    33124
1153516                                         don't    32888
2251707                                           m.a    32774
1896978                                           j.p    31531
3046787                                         r.h.s    31371
3880153                                           u.s    30551
2256301                                           m.j    30548
2839713                                      phys.rev    29991
1149248                                       doesn't    27899
745999                                            c.f    27462
325790                                            a.m    27234
747532                                            c.m    26912
3044023                                           r.a    25239
330218                                            a.v    25220
2822434                                          ph.d    25159
1901234                                           j.r    24912
1884834                                           j.c    24360
1012850                                           d.j    24303
747253                                            c.l    24147
1892456                                           j.l    23901
...                                               ...      ...
2491183                         multiferroicity.while        1
2491185                              multiferroics.=1        1
2491208  multifieldux20inflationaryux20predictions3.2        1
2491210       multifieldux20inflationux20formalism6.1        1
998661                                         cvp7.1        1
998659                                 cvp.algorithms        1
998650                                 cvoo.geomar.de        1
2491204                          multifield_galileons        1
2491186                               multiferroics.a        1
2491187                   multiferroics.isostructural        1
998720                       cvpr.2005.297recognizing        1
998721                              cvpr.2008.4587747        1
2491077                                    multienv.m        1
998722                              cvpr.2010.5539932        1
998785                       cvprlab.uniparthenope.it        1
998781                                      cvpr_yolo        1
998780                                  cvpr_tutorial        1
998772                                  cvpr2014.html        1
2491116                                   multif_rees        1
998735                               cvpr.2017.571are        1
998734                                   cvpr.2016.90        1
998733                                  cvpr.2016.494        1
998732                              cvpr.2015.7299155        1
998729                              cvpr.2015.7298654        1
998728                              cvpr.2015.7298640        1
998725                                   cvpr.2013.81        1
998724                                  cvpr.2013.446        1
998723                              cvpr.2010.5539938        1
998736                                     cvpr.zl.io        1
998749                           cvpr18_glimpseclouds        1

[1197426 rows x 2 columns]